In [224]:
import os
import gc  #回收内存，节约资源
import torch
import random
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from 深度学习入门.测试环境 import device

In [225]:
"""设置随机种子"""


def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [226]:
"""读取pt文件---一段pt文件就是一段语音（特征）"""


def load_features(path):
    features = torch.load(path)
    return features  #读取的pt文件的内容是张量(由t个39维的MFCC向量构成（简单理解就是：t个30维的帧）)

In [227]:
"""拼接frame"""

"""
传入的参数：
x[2],n=1
    right=x[2][-1].repeat(1,1)=x[2][-1]   (第二组的最后一个帧)#注意我们传入的是第二组的x，所以这个操作的x[-1]也是属于第二组的
    left=x[2][1:]   (第二组从标号为1的帧开始，所有的帧)
    cat((x[2][1:],x[2][-1]),dim=0)
x[0],n=-1
    left=x[0][0].repeat(1,1)=x[0][0]
    right=x[0][:-1]    (第零组的除了最后一个帧以外的其他帧)
    cat((x[0][0],x[0][:-1]),dim=0)
"""


def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x
    return torch.cat((left, right), dim=0)

In [228]:
"""
为了提高预测的准确率，进行左右帧的拼接
拼接frame
"""

"""
x:[T,39]
"""


def concat_features(x, concat_num):
    #x:t个39维的MFCC向量构成的
    #concat_num:拼接的数目（多少个帧需要拼接在一起）
    assert concat_num % 2 == 1  #需要是奇数，因为需要左边和右边都进行拼接
    #特殊情况:只有一个帧，就不用进行拼接
    if concat_num < 2:
        return x

    seq_len = x.size(0)  #frame的个数
    features_dim = x.size(1)  #每个frame的维度--39

    x = x.repeat(1, concat_num)  #沿着第一维（即竖直方向重复了concat_num次）
    """
    假设concat_num=3
    seq_len=T,features_dim=39
    view:[seq_len, concat_num, features_dim]   重构  [T,3,39]
    permute:[concat_num,seq_len,features_dim]        [3,T,39]
    [组号，帧序号，帧内数据序号]
    """
    # concat_num: 代表每个时间步上，我们拼接了多少个相邻的帧（包括当前帧）
    x = x.view(seq_len, concat_num, features_dim).permute(1, 0, 2)

    mid = (concat_num // 2)  #取中间帧
    """
    mid=3//2=1
    r_idx=range(1,2)=1 #左闭右开
    
    x[2,:]=shift(x[2],1)
    x[0,:]=shift(x[0],-1)
    """

    for r_idx in range(1, mid + 1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)
    """[seq_len,concat_num,features_dim]=[T,3,39]"""
    return x.permute(1, 0, 2).view(seq_len,
                                   concat_num * features_dim)  #seq_len向量个数，concat_num * features_dim：将拼接在一起的相邻帧的所有特征维度加起来得到的总维度

In [229]:
"""预处理"""
"""
split:读取保存了文件名的文本文档；区分当前的数据是测试集还是非测试集
feat_dir:pt文件保存路径
phone_path:3个文本文档所在的路径
concat_nframes:多少个帧拼接在一起
train_ratio=0.8:trian和val数据集的比例
train_val_seed:随机种子数
"""


def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1999):
    class_num = 41  #音素41个类别
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()  #逐行读取
        for line in phone_file:
            line = line.strip('\n').split(' ')  #将换行符删除，使用空格进行隔开
            label_dict[line[0]] = [int(p) for p in line[1:]]  #第零个元素作为字典中的key,其余的作为value
            # 展开形式
            # temp = []
            # for p in line[1:]:
            #     temp.append(int(p))
    """划分数据集"""
    if split == 'train' or split == 'val':
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        precent = int(len(usage_list) * train_ratio)  #划分的索引
        if split == 'train':
            usage_list = usage_list[:precent]
        elif split == 'val':
            usage_list = usage_list[precent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError("Invalid \'split\' argument for dataset: PhoneDataset!")

    usage_list = [line.strip('\n') for line in usage_list]  #删除换行符
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(
        len(usage_list)))  #类别，当前的数据集，语音文件
    max_len = 5000000
    x = torch.empty(max_len, 39 * concat_nframes)  #concat_nframes：帧数
    if mode != 'test':  #训练或验证环节需要用到label值
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):  #usage_list：保存的文件名 #enumerate相当于给数据加上索引
        feat = load_features(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)  #当前pt文件有多少个帧构成的
        feat = concat_features(feat, concat_nframes)  #拼接帧
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])

        x[idx: idx + cur_len] = feat

        if mode != 'test':
            y[idx: idx + cur_len] = label

        idx += cur_len
    x = x[:idx, :]
    if mode != 'test':
        y = y[:idx]
    print(f'[INFO] {split} set')
    print(x.shape)
    if mode != 'test':
        print(y.shape)
        return x, y
    else:
        return x

In [230]:
"""Dateset"""


class LibriDataset(Dataset):
    def __init__(self, x, y=None):
        self.feature = x
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.feature[idx], self.label[idx]
        else:
            return self.feature[idx]

    def __len__(self):
        return len(self.feature)

In [231]:
"""Model"""


class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


"""
*args：表示传入不定个数的参数
for _ in range(hidden_layers): #=填充多少个BasicBlock
    temp.append([BasicBlock(hidden_dim, hidden_dim)])
"""


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_dim=256, hidden_layers=1):  #hidden_layers=1隐藏层的个数
        super(Classifier, self).__init__()
        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),  #输入层
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],  #隐藏层
            nn.Linear(hidden_dim, output_dim)  #输出层
        )

    def forward(self, x):
        x = self.fc(x)
        return x


In [232]:
'''参数定义'''
device = 'cuda' if torch.cuda.is_available() else 'cpu'

concat_nframes = 3  #多少帧拼接在一起。必须为奇数个，n=2k+1
train_ratio = 0.8

config = {
    'seed': 0,
    'batch_size': 512,
    'n_epoch': 5,
    'learning_rate': 0.0001,
    'model_path': '/model.ckpt'
}

input_dim = 39 * concat_nframes  #拼接后的一个帧的维度
hidden_layers = 3
hidden_dim = 256

In [233]:
'''训练过程'''


def train_process(train_set, val_set, train_loader, val_loader, config, model, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
    best_acc = 0.0
    n_epoch = config['n_epoch']

    for epoch in range(n_epoch):
        train_acc = 0.0
        train_loss = 0.0
        '''训练模式'''
        model.train()
        for i, batch in enumerate(tqdm(train_loader)):
            '''batch:dataloader中一个批次的数据，以及对应的labels'''
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            '''torch.max会返回两个值，具体的value，以及value所在的index'''
            _, train_pred = torch.max(outputs, dim=1)  #得到可能性最高的类别编号 #dim=1比较行
            train_acc += (train_pred.detach() == labels.detach()).sum().item()  #detach()阻断反向传播，比较数据大小
            train_loss += loss.item()

        '''验证模式'''

        if (len(val_set) > 0):
            val_acc = 0.0
            val_loss = 0.0
            model.eval()
            with torch.no_grad():
                for i, batch in enumerate(tqdm(val_loader)):
                    features, labels = batch
                    features = features.to(device)
                    labels = labels.to(device)
                    outputs = model(features)

                    loss = criterion(outputs, labels)
                    _, val_pred = torch.max(outputs, dim=1)
                    val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
                    val_loss += loss.item()

                print('[{:03d}/{:03d}] Train Acc : {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss : {:3.6f}'.format(
                    epoch + 1, n_epoch, train_acc / len(train_set), train_loss / len(train_loader),
                    val_acc / len(val_set), val_loss / len(val_loader)))

                #保存最优模型
                if val_acc > best_acc:
                    best_acc = val_acc
                    torch.save(model.state_dict(), config['model_path'])
                    print("saving model with acc:{:3.6}".format(best_acc / len(train_loader)))
        else:
            print("[{:03d}/{:03d}] Train Acc : {:3.6f} Loss: {:3.6f} ".format(epoch + 1, n_epoch,
                                                                              train_acc / len(train_set),
                                                                              train_loss / len(train_loader)))

    if len(val_set) == 0:  #假设没有划分验证集。那最后保存的模型就是跑完最后一趟的epoch的模型
        torch.save(model.state_dict(), config['model_path'])
        print("saving model at last epoch")
    '''释放内存，回收资源'''
    del train_loader, val_loader
    gc.collect()


In [234]:
'''训练准备'''
set_seed(config['seed'])
train_x, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone',
                                   concat_nframes=concat_nframes, train_ratio=train_ratio)
val_x, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone',
                               concat_nframes=concat_nframes, train_ratio=train_ratio)

train_set = LibriDataset(train_x, train_y)
val_set = LibriDataset(val_x, val_y)

del train_x, train_y, val_x, val_y
gc.collect()

train_dataloader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
val_dataloader = DataLoader(val_set, batch_size=config['batch_size'], shuffle=True, pin_memory=True)


[Dataset] - # phone classes: 41, number of utterances for train: 2743


0it [00:00, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_23740\3898882972.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  features = torch.load(path)
2743it [00:05,

[INFO] train set
torch.Size([1692441, 117])
torch.Size([1692441])
[Dataset] - # phone classes: 41, number of utterances for val: 686


686it [00:01, 574.12it/s]


[INFO] val set
torch.Size([424353, 117])
torch.Size([424353])


In [235]:
'''开始训练'''
print(f'DEVICE: {device}')
model = Classifier(input_dim=input_dim, hidden_dim=256, hidden_layers=hidden_layers).to(device)
train_process(train_set, val_set, train_dataloader, val_dataloader, config, model, device)


DEVICE: cuda


100%|██████████| 829/829 [00:05<00:00, 138.36it/s]


[001/005] Train Acc : 0.443827 Loss: 1.964963 | Val Acc: 0.476436 loss : 1.805017
saving model with acc:61.1546


100%|██████████| 829/829 [00:06<00:00, 123.95it/s]


[002/005] Train Acc : 0.493030 Loss: 1.742287 | Val Acc: 0.499207 loss : 1.714210
saving model with acc:64.0774


...

[003/005] Train Acc : 0.510005 Loss: 1.670788 | Val Acc: 0.509632 loss : 1.662607
saving model with acc:65.4156


100%|██████████| 829/829 [00:04<00:00, 173.12it/s]


[004/005] Train Acc : 0.520666 Loss: 1.623920 | Val Acc: 0.517296 loss : 1.629743
saving model with acc:66.3993


100%|██████████| 829/829 [00:04<00:00, 174.77it/s]

[005/005] Train Acc : 0.528172 Loss: 1.590537 | Val Acc: 0.524408 loss : 1.600099
saving model with acc:67.3122


In [236]:
'''test预测'''


def predict(model, test_loader):
    test_acc = 0.0
    test_length = 0.0
    pred = np.array([], dtype=np.int32)

    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(tqdm(test_loader)):
            features = batch
            features = features.to(device)
            outputs = model(features)

            _, test_pred = torch.max(outputs, dim=1)

            pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

    return pred


'''保存预测结果'''


def save_pred(preds, file):
    with open(file, 'w') as f:
        f.write('Id,Class\n')
        for i, y in enumerate(preds):
            f.write('{},{}\n'.format(i, y))


test_x = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone',
                         concat_nframes=concat_nframes)
test_set = LibriDataset(test_x, None)
test_loader = DataLoader(test_set, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers).to(device)
model.load_state_dict(torch.load(config['model_path']))

preds = predict(model, test_loader)
save_pred(preds, 'prediction.csv')

[Dataset] - # phone classes: 41, number of utterances for test: 857


0it [00:00, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_23740\3898882972.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  features = torch.load(path)
857it [00:01, 

[INFO] test set
torch.Size([527364, 117])


100%|██████████| 1031/1031 [00:05<00:00, 201.84it/s]
